In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('donald_trump_twitter.csv', lineterminator="\n")

In [98]:
data.sample(5)

,created_at,favorite_count,id_str,is_retweet,retweet_count,source,text,local_time
13201,Tue Apr 18 10:46:19 +0000 2017,62665,854284952539725828,False,16046,Twitter for iPhone,Republicans must get out today and VOTE in Geo...,2017-04-18 06:46:19
31895,Sat Nov 16 11:43:53 +0000 2013,168,401676995668746240,False,252,Twitter for Android,"""@swetasamota: “Be tough, be smart, be person...",2013-11-16 06:43:53
43515,Fri Sep 02 18:27:03 +0000 2011,29,109693836657295361,False,209,TweetDeck,Fast and Furious put semi-automatics in the ha...,2011-09-02 14:27:03
566,Tue Dec 10 23:22:24 +0000 2019,0,1204541906535043072,True,4613,Twitter for iPhone,RT @RepArrington: After a year of needless del...,2019-12-10 18:22:24
4985,Mon Jul 01 00:47:37 +0000 2019,150504,1145494128190480384,False,37386,Twitter for iPhone,https://t.co/ctb5YJL1Vb,2019-06-30 20:47:37


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43981 entries, 0 to 43980
Data columns (total 7 columns):
created_at        43981 non-null object
favorite_count    43981 non-null int64
id_str            43981 non-null int64
is_retweet        43923 non-null object
retweet_count     43981 non-null int64
source            43981 non-null object
text              43981 non-null object
dtypes: int64(3), object(4)
memory usage: 2.3+ MB


In [5]:
data.describe()

,favorite_count,id_str,retweet_count
count,43981.000000,4.398100e+04,43981.000000
mean,22005.092108,6.612991e+17,6600.277370
std,41833.312001,3.209879e+17,11065.391745
min,0.000000,1.698309e+09,0.000000
25%,19.000000,3.777394e+17,26.000000
50%,126.000000,6.065359e+17,569.000000
75%,24046.000000,9.568500e+17,11115.000000
max,879647.000000,1.208951e+18,369530.000000


In [6]:
data['is_retweet'].value_counts()

False    42167
True      1756
Name: is_retweet, dtype: int64

## 1. 数据清洗
1.1 时间数据清洗

In [23]:
data['local_time'] = pd.to_datetime(data['created_at']).dt.tz_convert('US/Eastern').dt.strftime('%Y/%m/%d %H:%M:%S')

In [25]:
data['local_time'] = pd.to_datetime(data['local_time'])

## 2. 问题

2.1 特朗普使用推特基本情况  
2.2 最不受特朗普待见的美国主流媒体是什么？  
2.3 谁是被特朗普怼得最多的人？  
2.4 特朗普的贬义词词库  
2.5 特朗普的自我营销  
2.6 特朗普发推的固定模式  
2.7 跟中国有关的推特  

### 2.1 特朗普使用推特基本情况

In [50]:
by_year = data.groupby(data['local_time'].dt.year).size()
by_year

local_time
2009      56
2010     142
2011     774
2012    3531
2013    8144
2014    5792
2015    7530
2016    4223
2017    2602
2018    3572
2019    7615
dtype: int64

In [51]:
data['local_time'].describe()

count                   43981
unique                  43634
top       2018-07-15 09:33:15
freq                        5
first     2009-05-04 14:54:25
last      2019-12-22 22:22:23
Name: local_time, dtype: object

In [684]:
47000/3884

12.100926879505664

In [177]:
from pyecharts import Line


line = Line("特朗普每年发推数量", "2009年5月4日至2019年12月22日 单位: 条", width=700)
line.add("", list(map(str, list(by_year.index))), list(by_year.values), 
         is_smooth=True, legend_text_size=18,xaxis_label_textsize=16,
         yaxis_label_textsize=18, yaxis_min=0, mark_point=['max', 'min'], 
         mark_point_symbol='pin', mark_line=['average'])
line

In [83]:
by_year.sum()

43981

In [117]:
# 经过验证，这个is_retweet字段并没有完全记录是否转推，也就是说这个字段不准确
data['is_retweet'].value_counts()

False    42167
True      1756
Name: is_retweet, dtype: int64

In [120]:
data[data['local_time'] > pd.to_datetime('2019-01-01 00:00:00')].shape[0]

7615

In [121]:
# 2019年平均每天发推21条
7615/(365-9)

21.390449438202246

In [125]:
# 自18日被众议院弹劾以来，共发推206条
data[data['local_time'] > pd.to_datetime('2019-12-18 00:00:00')].shape[0]

206

[David Robinson](http://varianceexplained.org/r/trump-tweets/)于2016-08-09通过数据分析发现Trump自己只用Android端发推特

In [127]:
data[data['local_time'] < pd.to_datetime('2016-08-09')]['source'].value_counts()

Twitter for Android         13814
Twitter Web Client          11996
Twitter for iPhone           1635
TweetDeck                     481
TwitLonger Beta               405
Instagram                     133
Facebook                      105
Twitter for BlackBerry         97
Mobile Web (M5)                56
Twitter Ads                    48
Twitlonger                     23
Vine - Make a Scene            10
Twitter QandA                  10
Periscope                       6
Neatly For BlackBerry 10        5
Twitter for iPad                2
Twitter Mirror for iPad         1
Twitter for Websites            1
Name: source, dtype: int64

In [134]:
data[data['source']=='Twitter for Android'].groupby(data['local_time'].dt.year).size()

local_time
2013    3708
2014    3549
2015    5173
2016    1834
2017     281
dtype: int64

In [135]:
data[(data['local_time'] > pd.to_datetime('2017-01-01'))&
    (data['source'] == 'Twitter for Android')].groupby(data['local_time'].dt.month).size()

local_time
1    146
2    109
3     26
dtype: int64

也就是说，自2017年3月（本人只用Android发推该文章被大量报道之后）以后，川普再也没有用Android发过推，从此之后，绝大部分推特都是iPhone端发出的

In [138]:
data[data['local_time'] > pd.to_datetime('2017-04-01')]['source'].value_counts()

Twitter for iPhone      12867
Media Studio              154
Twitter Media Studio      133
Twitter Web Client         47
Twitter for iPad           38
Twitter Ads                31
Twitter Web App             4
Name: source, dtype: int64

In [139]:
data['source'].value_counts()

Twitter for iPhone          15468
Twitter for Android         14545
Twitter Web Client          12182
TweetDeck                     483
TwitLonger Beta               405
Media Studio                  156
Instagram                     133
Twitter Media Studio          133
Facebook                      105
Twitter Ads                    97
Twitter for BlackBerry         97
Twitter for iPad               60
Mobile Web (M5)                56
Twitlonger                     23
Vine - Make a Scene            10
Twitter QandA                  10
Periscope                       7
Neatly For BlackBerry 10        5
Twitter Web App                 4
Twitter Mirror for iPad         1
Twitter for Websites            1
Name: source, dtype: int64

In [168]:
river_data = data[data['source'].isin(['Twitter for iPhone',
                         'Twitter for Android',
                         'Twitter Web Client'])].groupby(['source',data['local_time'].dt.year]).size().reset_index()
river_data

,source,local_time,0
0,Twitter Web Client,2009,56
1,Twitter Web Client,2010,142
2,Twitter Web Client,2011,277
3,Twitter Web Client,2012,3099
4,Twitter Web Client,2013,4254
5,Twitter Web Client,2014,2201
6,Twitter Web Client,2015,1757
7,Twitter Web Client,2016,340
8,Twitter Web Client,2017,49
9,Twitter Web Client,2018,6


In [169]:
river_data.rename(columns={0:'number'}, inplace=True)

In [170]:

river_data['local_time'] = river_data['local_time'].astype(str)
river_data = river_data[['local_time', 'number', 'source']]

In [171]:
river_data = river_data.values.tolist()

In [172]:
river_data

[['2009', 56, 'Twitter Web Client'],
 ['2010', 142, 'Twitter Web Client'],
 ['2011', 277, 'Twitter Web Client'],
 ['2012', 3099, 'Twitter Web Client'],
 ['2013', 4254, 'Twitter Web Client'],
 ['2014', 2201, 'Twitter Web Client'],
 ['2015', 1757, 'Twitter Web Client'],
 ['2016', 340, 'Twitter Web Client'],
 ['2017', 49, 'Twitter Web Client'],
 ['2018', 6, 'Twitter Web Client'],
 ['2019', 1, 'Twitter Web Client'],
 ['2013', 3708, 'Twitter for Android'],
 ['2014', 3549, 'Twitter for Android'],
 ['2015', 5173, 'Twitter for Android'],
 ['2016', 1834, 'Twitter for Android'],
 ['2017', 281, 'Twitter for Android'],
 ['2012', 1, 'Twitter for iPhone'],
 ['2013', 8, 'Twitter for iPhone'],
 ['2014', 1, 'Twitter for iPhone'],
 ['2015', 412, 'Twitter for iPhone'],
 ['2016', 1957, 'Twitter for iPhone'],
 ['2017', 2103, 'Twitter for iPhone'],
 ['2018', 3498, 'Twitter for iPhone'],
 ['2019', 7488, 'Twitter for iPhone']]

In [174]:
# 该图不准确
from pyecharts import ThemeRiver


tr = ThemeRiver("主题河流图示例图")
tr.add(['Twitter for iPhone',
        'Twitter for Android',
        'Twitter Web Client'], 
       [['2009-05-04', 56, 'Twitter Web Client'],
 ['2010', 142, 'Twitter Web Client'],
 ['2011', 277, 'Twitter Web Client'],
 ['2012', 3099, 'Twitter Web Client'],
 ['2013', 4254, 'Twitter Web Client'],
 ['2014', 2201, 'Twitter Web Client'],
 ['2015', 1757, 'Twitter Web Client'],
 ['2016', 340, 'Twitter Web Client'],
 ['2017', 49, 'Twitter Web Client'],
 ['2018', 6, 'Twitter Web Client'],
 ['2019-12-22', 1, 'Twitter Web Client'],
 ['2013', 3708, 'Twitter for Android'],
 ['2014', 3549, 'Twitter for Android'],
 ['2015', 5173, 'Twitter for Android'],
 ['2016', 1834, 'Twitter for Android'],
 ['2017', 281, 'Twitter for Android'],
 ['2012', 1, 'Twitter for iPhone'],
 ['2013', 8, 'Twitter for iPhone'],
 ['2014', 1, 'Twitter for iPhone'],
 ['2015', 412, 'Twitter for iPhone'],
 ['2016', 1957, 'Twitter for iPhone'],
 ['2017', 2103, 'Twitter for iPhone'],
 ['2018', 3498, 'Twitter for iPhone'],
 ['2019-12-22', 7488, 'Twitter for iPhone']], is_label_show=False,
      is_legend_show=True)
tr

In [178]:
# 每天什么时候发推？
by_hour = data[data['local_time'] > pd.to_datetime('2019/1/1 00:00:00')].groupby(data['local_time'].dt.hour).size()
# by_hour = data.groupby(data['local_time'].dt.hour).size()

In [179]:
by_hour

local_time
0     189
1      24
2      18
3      13
4      14
5      31
6     338
7     763
8     789
9     614
10    483
11    298
12    267
13    273
14    301
15    329
16    296
17    375
18    410
19    351
20    327
21    290
22    393
23    429
dtype: int64

In [187]:
# 注: 时间为美国东部时间，即白宫所在地华盛顿时间
from pyecharts import Bar

bar = Bar("特朗普每日发推规律", "2019年1月1日至2019年12月22日", width=800)
bar.add("", list(by_hour.index), list(by_hour.values), is_stack=True, is_label_show=True,
       bar_category_gap='40%', label_color = ['#130f40'], label_text_size=12,
       legend_text_size=12,xaxis_label_textsize=18,yaxis_label_textsize=18)
bar

### 2.2 最不受特朗普待见的美国主流媒体是什么？

In [189]:
data['text'].sample(5)

24633    "@marino_joanna: DonaldTrump you would have my...
34371    Congratulations to @KingJames on winning Athle...
30943       Negotiation: It is persuasion more than power.
36731    All eyes are on @TigerWoods @The_Masters.  He’...
2589     Gerry Baker of @WSJatLarge  “Do you think what...
Name: text, dtype: object

In [255]:
us_media = pd.read_csv('us_media.csv')

In [256]:
medias = [m.lower() for m in list(us_media['media'])]

In [297]:
medias = ['fortune magazine',
 'national review',
 'politico',
 'los angeles times',
 'guardian',
 'usa today',
 'new hampshire union leader',
 'esquire magazine',
 'cnet',
 'gq magazine',
 'weekly standard',
 'hbo',
 'tmz',
 'npr',
 'los angeles daily',
 'bloomberg',
 'the view',
 'univision',
 'newsweek',
 'wall street journal',
 'techmeme',
 'huffington post',
 'meet the press',
 'cnn',
 'fox',
 'national journal',
 'msnbc',
 'bbc',
 'buzzfeed',
 'cnbc',
 'des moines register ',
 'time magazine',
 'vanity fair',
 'ny mag',
 'cbs',
 'ny post',
 'ny daily',
 "o'reilly factor",
 'espn',
 'charlie hebdo',
 'associated press',
 'morning joe',
 'deadspin',
 'time magazine ',
 'washington post',
 'palm beach post',
 'rolling stone',
 'nbc',
 'ny times|new york times|nytimes',
 'abc',
 'huffpost']

In [299]:
mentioned_count = [data['text'].str.lower().str.contains(media).sum() for media in medias]

In [300]:
us_media['mentioned_count'] = mentioned_count

In [301]:
media_top10 = us_media.sort_values(by='mentioned_count', ascending=False)[:10]
media_top10

,media,mentioned_count
24,Fox,1772
47,NBC,1003
23,CNN,585
48,NY Times|New York Times,326
49,ABC,173
29,CNBC,109
34,CBS,108
13,NPR,91
26,MSNBC,78
44,Washington Post,76


In [302]:
media_top10['media'].values

array(['Fox', 'NBC', 'CNN', 'NY Times|New York Times', 'ABC', 'CNBC',
       'CBS', 'NPR', 'MSNBC', 'Washington Post'], dtype=object)

In [303]:
# 被特朗普在推特中提及最多的媒体
from pyecharts import Bar

bar = Bar("特朗普在推特中提及最多的媒体", "2009年5月4日至2019年12月22日", width=800)
bar.add("", ['Fox', 'NBC', 'CNN', 'New York Times', 'ABC', 'CNBC', 'CBS', 'NPR', 'MSNBC', 'Washington Post'], 
        list(media_top10['mentioned_count'].values), is_stack=True, is_label_show=True,
       bar_category_gap='40%', label_color = ['#130f40'], label_text_size=12,xaxis_rotate=20,
       legend_text_size=16,xaxis_label_textsize=16,yaxis_label_textsize=18)
bar

In [304]:
stop_words_en = []
with open('stop_words_en.txt', 'r') as f:
    stop_words_en = [i.strip() for i in f.readlines()]
    
stop_words_en = list(set(stop_words_en))
stop_words_en.append('')
stop_words_en.extend(list(map(str,list(range(10)))))
stop_words_en.extend(['http', 'https', 'amp;', "Don't", 'RT'])
stop_words_en.extend(list('ABCDEFGHIJKLMMOPQRSTUVWXYZ'))
stop_words_en.extend(list('abcdefghijklmnopqrstuvwxyz'))

In [305]:
from collections import Counter
from pyecharts import WordCloud


def get_word_cloud(text_column):
    text = ' '.join(list(text_column))
    r =  "【.*?】+|\\《.*?》+|\\#.*?#+|[.!/_,$&%^*()<>+""'?@|:~{}#]+|[——！，。=？、：“”‘’￥……（）《》【】\"]"
    for i in list(r):
        text = text.replace(i, ' ')
    
    split_text = text.split(' ')
    text_list = [i for i in split_text if i not in stop_words_en]
    
    c = Counter()
    c = Counter(text_list)
    wc_data = pd.DataFrame({'word':list(c.keys()), 'counts':list(c.values())}).sort_values(by='counts', ascending=False).head(70)
    
    wordcloud = WordCloud(width=1000, height=600)
    wordcloud.add("", wc_data['word'], wc_data['counts'], word_size_range=[20, 100])
    return wordcloud 

In [311]:
data.loc[data['text'].str.lower().str.contains('|'.join(medias)), 'text'].shape

(4310,)

In [315]:
data[(data['text'].str.lower().str.contains('nbc'))&
    data['text'].str.lower().str.contains('fake news')]['id_str'].sample(5)

3259     1172858577951821825
13318     844886082663698436
10859     966650397002813440
11408     935844881825763328
9182     1030779412973846529
Name: id_str, dtype: int64

In [316]:
data[(data['text'].str.lower().str.contains('cnn'))&
    data['text'].str.lower().str.contains('fake news')]['id_str'].sample(5)

7648     1078431231262502912
8382     1054142516881248258
6637     1110714893039267840
13330     843803115044454402
11953     914094625488502784
Name: id_str, dtype: int64

In [308]:
get_word_cloud(data.loc[data['text'].str.lower().str.contains('|'.join(medias)), 'text'])

In [309]:
# 一共发了612条推特抨击Fake News！
data['text'].str.lower().str.contains('fake news').sum()

612

In [686]:
1003/47000*100

2.1340425531914895

In [310]:
get_word_cloud(data.loc[data['text'].str.lower().str.contains('fake news'), 'text'])

### 2.3 谁是被特朗普怼得最多的人？

In [328]:
# 首先计算最近常出现的名词
import nltk

In [329]:
text = ' '.join(list(data['text']))
r =  "【.*?】+|\\《.*?》+|\\#.*?#+|[.!/_,$&%^*()<>+""'?@|:~{}#]+|[——！，。=？、：“”‘’￥……（）《》【】\"]"

for i in list(r):
    text = text.replace(i, ' ')

tokens = nltk.word_tokenize(text.lower())
text = nltk.Text(tokens)
tags = nltk.pos_tag(text)

In [330]:
len(tags)

918760

In [331]:
tags[:10]

[('rt', 'NN'),
 ('andypuzder', 'NN'),
 ('it', 'PRP'),
 ('s', 'VBZ'),
 ('all', 'DT'),
 ('but', 'CC'),
 ('impossible', 'JJ'),
 ('to', 'TO'),
 ('look', 'VB'),
 ('at', 'IN')]

In [332]:
tagged_text = pd.DataFrame(tags, columns=['word', 'pos'])

In [338]:
# NN  名词单数
# NNS 名词复数
# NNP 专有名词
# NNPS 专有名词复数
tagged_text[tagged_text['pos'].isin(['NN', 'NNS', 'NNP', 'NNPS'])]['word'].value_counts()[:50]

t                  10199
co                  9976
realdonaldtrump     7490
i                   6863
https               5023
trump               4502
president           3430
http                3345
amp                 3224
rt                  2984
s                   2842
people              2627
thank               2181
thanks              2107
country             1968
time                1652
today               1417
news                1227
democrats           1174
donald              1002
job                  992
obama                979
years                969
media                924
day                  884
world                850
way                  844
vote                 832
america              828
deal                 823
jobs                 810
border               798
house                750
nothing              738
night                713
china                702
states               665
show                 653
year                 648
state                643


In [339]:
tagged_text[tagged_text['pos'].isin(['NN', 'NNS', 'NNP', 'NNPS'])]['word'].value_counts()[50:100]

campaign                 546
economy                  510
golf                     503
m                        492
trump2016                488
trade                    488
record                   486
history                  484
things                   482
tomorrow                 482
congratulations          481
foxandfriends            448
tax                      447
party                    446
everyone                 444
cnn                      442
republicans              440
course                   440
poll                     438
security                 438
don                      433
apprentice               427
morning                  424
book                     418
please                   415
office                   414
dems                     412
crime                    410
york                     408
success                  405
wall                     400
americans                385
debate                   381
others                   374
bill          

In [341]:
tagged_text[tagged_text['pos'].isin(['NN', 'NNS', 'NNP', 'NNPS'])]['word'].value_counts()[100:150]

immigration       335
life              335
hunt              332
times             331
week              330
witch             329
watch             327
support           327
family            325
speech            325
fact              325
honor             325
government        323
administration    322
nation            320
numbers           318
person            315
law               309
leadership        308
leader            304
women             303
impeachment       299
washington        298
gop               296
something         288
mexico            287
meeting           285
countries         285
season            284
anything          283
john              279
hotel             277
everything        270
lot               267
mueller           266
iran              265
fbi               265
democrat          263
whitehouse        262
taxes             262
p                 261
governor          260
mittromney        259
cont              258
celebrity         257
anyone    

In [343]:
mention_list = ['obama', 'democrats|dems|democrat', 'clinton', 'congress', 'hillary', 'pelosi']

In [344]:
mention_counts = [data['text'].str.lower().str.contains(i).sum() for i in mention_list]

In [345]:
mention_counts

[2847, 2019, 733, 624, 1041, 250]

In [352]:
bar = Bar("特朗普在推特中提及最多的人/对象(除媒体外)", "2009年5月4日至2019年12月22日", width=600)
bar.add("", ['奥巴马', '民主党', '希拉里','克林顿', '国会', '佩洛西'], 
        [2847, 2019, 1041, 733, 624, 250], is_stack=True, is_label_show=True,
       bar_category_gap='40%', label_color = ['#130f40'], label_text_size=12,
       legend_text_size=16,xaxis_label_textsize=18,yaxis_label_textsize=18)
bar

In [688]:
2847/43981*100

6.473249812418999

In [354]:
for i in mention_list:
    data.loc[data['text'].str.lower().str.contains(i), 'mentioned'] = i

In [356]:
# 提及次数时间变化
data[data['mentioned'].isin(mention_list)].groupby(['mentioned',data['local_time'].dt.year]).size().reset_index()

,mentioned,local_time,0
0,clinton,2012,15
1,clinton,2013,4
2,clinton,2014,10
3,clinton,2015,24
4,clinton,2016,115
5,clinton,2017,27
6,clinton,2018,34
7,clinton,2019,42
8,congress,2011,9
9,congress,2012,17


### 2.4 特朗普的贬义词词库

In [393]:
# JJ  形容词
# JJR 比较级的形容词
# JJS 最高级的形容词
adjective_word = tagged_text[tagged_text['pos'].isin(['JJ', 'JJR', 'JJS'])]['word'].value_counts()
adjective_word[:10]

great    6326
t        3111
new      1966
i        1934
s        1763
big      1628
good     1583
many     1373
u        1362
more     1092
Name: word, dtype: int64

In [394]:
adjective_word = adjective_word.to_frame().reset_index()
adjective_word.rename(columns={'index': 'words', 'word':'word_counts'}, inplace=True)

In [395]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [396]:
sid = SentimentIntensityAnalyzer()

In [409]:
def get_sent(text):
    sentiment = sid.polarity_scores(text)
    if sentiment['compound'] == 0:
        return 'neu'
    else:
        return max(sentiment, key=sentiment.get)

In [410]:
adjective_word['sentiment'] = adjective_word['words'].apply(get_sent)

In [412]:
adjective_word[:50]

,words,word_counts,sentiment
0,great,6326,pos
1,t,3111,neu
2,new,1966,neu
3,i,1934,neu
4,s,1763,neu
5,big,1628,neu
6,good,1583,pos
7,many,1373,neu
8,u,1362,neu
9,more,1092,neu


In [413]:
neg_word = adjective_word[adjective_word['sentiment']=='neg']

In [417]:
neg_word[:50]

,words,word_counts,sentiment
16,bad,856,neg
22,fake,665,neg
41,hard,396,neg
47,illegal,351,neg
50,wrong,325,neg
51,tough,324,neg
52,terrible,320,neg
71,sad,240,neg
81,worst,205,neg
84,low,201,neg


In [690]:
data[data['text'].str.lower().str.contains('bad')]['id_str'].sample(5)

18844    666719680179085313 
13791    815185071317676033 
9268     1029002242932834310
21168    621669173534584833 
10179    1000152863035805696
Name: id_str, dtype: int64

In [419]:
neg_word.shape[0]

529

In [421]:
wordcloud = WordCloud(width=1000, height=600)
wordcloud.add("", neg_word[:80]['words'], neg_word[:80]['word_counts'], word_size_range=[20, 100])
wordcloud 

### 2.5 特朗普的自我营销

In [434]:
adjective_word['sentiment'].value_counts()

neu    11955
pos      534
neg      529
Name: sentiment, dtype: int64

In [430]:
top10_adj = adjective_word[adjective_word['sentiment'].isin(['neg', 'pos'])][:11]

In [431]:
# united不能算，所以取11个
top10_adj

,words,word_counts,sentiment
0,great,6326,pos
6,good,1583,pos
11,best,1002,pos
13,true,908,pos
16,bad,856,neg
21,united,677,pos
22,fake,665,neg
29,strong,532,pos
30,nice,531,pos
33,happy,502,pos


In [437]:
print(list(top10_adj['words']))

['great', 'good', 'best', 'true', 'bad', 'united', 'fake', 'strong', 'nice', 'happy', 'better']


In [438]:
print(list(top10_adj['word_counts']))

[6326, 1583, 1002, 908, 856, 677, 665, 532, 531, 502, 500]


In [439]:
bar = Bar("特朗普最常使用的10个带有感情色彩的形容词", "2009年5月4日至2019年12月22日", width=800)
bar.add("", ['great', 'good', 'best', 'true', 'bad', 'fake', 'strong', 'nice', 'happy', 'better'], 
        [6326, 1583, 1002, 908, 856, 665, 532, 531, 502, 500],
        is_stack=True, is_label_show=True,
       bar_category_gap='40%', label_color = ['#130f40'], label_text_size=12,xaxis_rotate=20,
       legend_text_size=16,xaxis_label_textsize=16,yaxis_label_textsize=18)
bar

In [458]:
pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)

data[(data['text'].str.lower().str.contains('i am|i will|my'))&
    (data['text'].str.lower().str.contains('best|highest|greatest|biggest'))]['text'].sample(5)

11198    On Taxes: “This is the biggest corporate rate cut ever, going back to the corporate income tax rate of roughly 80 years ago.This is a huge pro-growth stimulus for the economy. Every year the Obama WH overstated how the economy would grow. Now real economics and jobs.” @WSJ Report
43039    My new book #TimeToGetTough is the best present of the holiday season. A great gift for anyone who cares about this country.                                                                                                                                                            
5825     RT @realDonaldTrump: ....The only problem is that they know I am going to win (best economy &amp; employment numbers in U.S. history, &amp; much mo…                                                                                                                                    
14677    My supporters are the best! $18 million from hard-working people who KNOW what we can be again! Shatter the record: https

In [462]:
data[data['text'].str.lower().str.contains('i am the')]['text'].sample(5)

33097    "@dashinglis: @realDonaldTrump @robin1260 I'm in the states next week and whenever I am there I always buy Trump shirts and ties. I luv em"                                                                              
17056    I am the only Republican who will get large numbers of Dems and Indies (crossover). I will also get states that no other Republican can get.                                                                             
4168     ....mentally ill or deranged people. I am the biggest Second Amendment person there is, but we all must work together for the good and safety of our Country. Common sense things can be done that are good for everyone!
22485    I am the BEST builder, just look at what I've built. Hillary can't build. Republican candidates can't build. They don't have a clue!                                                                                     
22687    I am the only one that knows how to build cities - pols are all talk and no action.

### 2.6 特朗普发推的固定模式

In [489]:
import string


'|'.join(string.punctuation)

'!|"|#|$|%|&|\'|(|)|*|+|,|-|.|/|:|;|<|=|>|?|@|[|\\|]|^|_|`|{|||}|~'

In [517]:
(data['text'].str.extract(r'.*(!)$') == '!').sum()

0    12458
dtype: int64

In [520]:
(data['text'].str.extract(r'.*(\?)$') == '?').sum()

0    910
dtype: int64

In [521]:
(data['text'].str.extract(r'.*(\.\.\.)$') == '...').sum()

0    1114
dtype: int64

In [522]:
(data['text'].str.extract(r'.*(\.)$') == '.').sum()

0    8875
dtype: int64

In [526]:
# 把句子最后的字符选取出来
data['punctuation'] = data['text'].str.slice(start=-1)

In [528]:
data['punctuation'].value_counts()[:10]

!    12458
.    8875 
"    5653 
…    2836 
?    910  
s    862  
e    712  
n    603  
h    411  
t    356  
Name: punctuation, dtype: int64

In [698]:
12458/43981

0.2832586798844956

In [574]:
trump_twitter = data[-data['text'].str.contains('http')]

In [602]:
# 把最后一句选取出来
trump_twitter['last_sentence'] = trump_twitter['text'].str.replace(',|\?|:|!|#|-|"|–|\n|“|;|”', '.').apply(get_last_sentence)

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [615]:
trump_twitter['last_sentence'] = trump_twitter['last_sentence'].str.strip()

/root/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [623]:
trump_twitter['last_sentence'].sample(5)

11358    a Pelosi/Schumer Puppet                                                       
10393    Very dishonest and sick                                                       
10615    They should clean up and strengthen CNN and get back to honest reporting      
37734    the worst thing you can do in a negotiation is seem desperate to make the deal
9700                                                                                   
Name: last_sentence, dtype: object

In [693]:
trump_twitter['last_sentence'].value_counts()[:50]

                            3939
Thanks                      680 
Thank you                   537 
Trump2016                   239 
thanks                      176 
Enjoy                       172 
CelebApprentice             169 
Great                       167 
MAKE AMERICA GREAT AGAIN    125 
True                        120 
MakeAmericaGreatAgain       120 
…                           114 
So true                     70  
Think Like a Champion       70  
@realDonaldTrump            68  
Think Big                   65  
MAGA                        64  
)                           59  
@foxandfriends              56  
TimeToGetTough              52  
@FoxNews                    51  
I agree                     50  
Sad                         50  
Thx                         49  
M                           48  
S                           47  
Trump                       46  
Nice                        45  
Midas Touch                 44  
@nflcommish                 43  
thank you 

In [697]:
data[data['text'].str.endswith('Sad!')]['id_str']

3267     1172494650185326593
3344     1171429005251104769
4188     1159260904631836673
4359     1155792447429054465
4623     1151089269538217984
5247     1139164977271595010
5427     1135499002626154496
6437     1116911640568573952
9441     1023546197129224192
9937     1007972046666690561
10352    992354530510721025 
11152    949287555660500992 
11673    924400308834455552 
11778    920611770775064576 
12672    885081181980590084 
12879    875682853585129472 
12899    874576057579565056 
13148    857370270776086528 
13434    835325771858251776 
13444    834181712783560705 
13689    820255947956383744 
14915    772571293438840832 
15084    764998650598686721 
16273    727634574298255361 
16369    724445148910915584 
16771    710407581899079680 
16898    707258582501302272 
16990    705227593205878784 
17410    696782375863394304 
17742    690382564494839809 
18138    679538962948571137 
19315    659392248589627392 
27133    521731699403923456 
27425    516603779819786241 
28276    49418

In [631]:
get_word_cloud(trump_twitter['last_sentence'])

### 7. 跟中国有关的推特

In [632]:
data_china = data[data['text'].str.lower().str.contains('china|chinese')]

In [634]:
data_china.shape

(835, 11)

In [679]:
data_china.sample(5)

,created_at,favorite_count,id_str,is_retweet,retweet_count,source,text,local_time,mentioned,punctuation,last_sentence,sentiment
35973,Wed May 08 19:53:27 +0000 2013,25,332221731618029569,False,95,Twitter Web Client,Scary. Our military is a using a Chinese made satellite for North Africa command communications http://t.co/054Ext5c1A,2013-05-08 15:53:27,NaN,A,co/054Ext5c1A,neu
4039,Thu Aug 15 16:52:53 +0000 2019,80707,1162044502292537344,False,21570,Twitter for iPhone,"The Fake News Media is doing everything they can to crash the economy because they think that will be bad for me and my re-election. The problem they have is that the economy is way too strong and we will soon be winning big on Trade, and everyone knows that, including China!",2019-08-15 12:52:53,NaN,!,including China,neu
30457,Thu Mar 20 23:44:54 +0000 2014,446,446794542839500802,False,499,Twitter for Android,"We're spending a fortune looking for the lost plane with mostly Chinese passengers, and that's OK-but how much are Russia &amp; China spending?",2014-03-20 19:44:54,NaN,?,but how much are Russia &amp; China spending,neu
4491,Tue Jul 23 11:54:02 +0000 2019,59437,1153634372442107904,False,12110,Twitter for iPhone,"Farmers are starting to do great again, after 15 years of a downward spiral. The 16 Billion Dollar China “replacement” money didn’t exactly hurt!",2019-07-23 07:54:02,NaN,!,The 16 Billion Dollar China “replacement” money didn’t exactly hurt,neu
2796,Tue Oct 01 10:54:26 +0000 2019,81387,1178986524630802432,False,13594,Twitter for iPhone,Congratulations to President Xi and the Chinese people on the 70th Anniversary of the People’s Republic of China!,2019-10-01 06:54:26,NaN,!,Congratulations to President Xi and the Chinese people on the 70th Anniversary of the People’s Republic of China,neu


In [680]:
data.describe()

,favorite_count,id_str,retweet_count
count,43981.000000,4.398100e+04,43981.000000
mean,22005.092108,6.612991e+17,6600.277370
std,41833.312001,3.209879e+17,11065.391745
min,0.000000,1.698309e+09,0.000000
25%,19.000000,3.777394e+17,26.000000
50%,126.000000,6.065359e+17,569.000000
75%,24046.000000,9.568500e+17,11115.000000
max,879647.000000,1.208951e+18,369530.000000


In [681]:
data_china.describe()

,favorite_count,id_str,retweet_count
count,835.000000,8.350000e+02,835.000000
mean,34044.166467,6.548757e+17,8473.594012
std,44524.794590,4.132442e+17,10921.559679
min,0.000000,3.073512e+16,0.000000
25%,23.500000,2.466865e+17,123.000000
50%,186.000000,5.919852e+17,720.000000
75%,73941.500000,1.120445e+18,16816.500000
max,275392.000000,1.208936e+18,104633.000000
